# Register files from Census release 2023-11-13

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

In [ ]:
census_version = "2023-11-13"

In [ ]:
s3path = f"s3://cellxgene-data-public/cell-census/{census_version}/h5ads"
ln.UPath(s3path).view_tree()

In [ ]:
ln.track()

## Register files

In [ ]:
files = ln.File.from_dir(s3path)
ln.save(files)

In [ ]:
dataset = ln.Dataset(files, name="cellxgene-census", version=census_version)
dataset.save()

## Register metadata

Get all datasets and associated metadata using cellxgene REST API:

In [ ]:
import requests


def get_datasets_df_from_cxg():
    api_url_base = "https://api.cellxgene.cziscience.com"
    datasets_path = "/curation/v1/datasets"
    datasets_url = f"{api_url_base}{datasets_path}"
    headers = {"Content-Type": "application/json"}
    res = requests.get(url=datasets_url, headers=headers)
    res.raise_for_status()
    res_content = res.json()
    return res_content

In [ ]:
res_content = get_datasets_df_from_cxg()
len(res_content)

In [ ]:
res_content[0].keys()

In [ ]:
features = ln.Feature.lookup()
files = ln.File.filter(key__contains=census_version).all()

### collections, organisms

In [ ]:
is_collection = ln.ULabel.filter(name="is_collection").one()
collections = is_collection.children.all()
organisms = lb.Organism.filter().all()

for dataset_meta in res_content:
    file = files.filter(key__contains=dataset_meta["dataset_id"]).one_or_none()
    if file is None:
        continue
    # register collection
    collection = collections.filter(
        reference=dataset_meta["collection_id"]
    ).one_or_none()
    if collection is None:
        collection = ln.ULabel(
            name=dataset_meta["collection_name"],
            description=dataset_meta["collection_doi"],
            reference=dataset_meta["collection_id"],
            reference_type="collection_id",
        )
        collection.save()
        collection.parents.add(is_collection)
    file.labels.add(collection, feature=features.collection)

    # annotate with organism
    organism_ontology_ids = [i["ontology_term_id"] for i in dataset_meta["organism"]]
    organism_records = organisms.filter(ontology_id__in=organism_ontology_ids).list()
    # register new organisms
    if len(organism_records) == 0:
        bionty_source = lb.BiontySource.filter(entity="Organism", organism="all").one()
        for i in organism_ontology_ids:
            record = lb.Organism.from_bionty(ontology_id=i, bionty_source=bionty_source)
            record.save(parents=False)
            print(f"registered organism: {record}")
        organism_records = organisms.filter(
            ontology_id__in=organism_ontology_ids
        ).list()
    file.labels.add(organism_records, feature=features.organism)

### obs ontologies

In [ ]:
feature_names = [
    "self_reported_ethnicity",
    "development_stage",
    "cell_type",
    "assay",
    "tissue",
    "disease",
    "sex",
    "donor_id",
    "suspension_type",
]

from lamindb.dev._feature_manager import get_accessor_by_orm

ACCESSORS = get_accessor_by_orm(ln.File)
FEATURE_TO_ACCESSOR = {}
for name in feature_names:
    feature = getattr(features, name)
    accessor = ACCESSORS.get(feature.registries)
    orm = getattr(ln.File, accessor).field.model
    # TODO: ulabels are defined in the File model, improve this in LaminDB
    if orm == ln.File:
        orm = getattr(ln.File, accessor).field.related_model
    FEATURE_TO_ACCESSOR[name] = (accessor, orm)

In [ ]:
obs_featureset = ln.FeatureSet(features=[getattr(features, i) for i in feature_names])
obs_featureset.save()

obs_featureset.files.set(files, through_defaults={"slot": "obs"})

In [ ]:
# extra step to register uberon ontologies as developmental stages
def create_dv_record_from_uberon(ontology_id: str):
    tissue_record = lb.Tissue.from_bionty(ontology_id=ontology_id)
    dvs_record = lb.DevelopmentalStage(
        name=tissue_record.name,
        description=tissue_record.description,
        ontology_id=tissue_record.ontology_id,
        bionty_source_id=tissue_record.bionty_source_id,
    )
    dvs_record.save()


for id in [
    "UBERON:0018241",
    "UBERON:0034919",
    "UBERON:0007222",
    "UBERON:0000113",
    "UBERON:0007220",
    "UBERON:0007222",
]:
    create_dv_record_from_uberon(id)

In [ ]:
ontology_ids = {}
for name in feature_names:
    if name in ["donor_id", "suspension_type"]:
        continue
    allids = set()
    for i in res_content:
        if name in i:
            allids.update([(j["label"], j["ontology_term_id"]) for j in i[name]])

    ontology_ids[name] = allids

# register all ontology ids
for name, terms in ontology_ids.items():
    accessor, orm = FEATURE_TO_ACCESSOR.get(name)
    terms_ids = [i[1] for i in terms]
    records = orm.from_values(terms_ids, field="ontology_id")
    if len(records) > 0:
        ln.save(records)

In [ ]:
# register the non-validated terms
bionty_source_ds_mouse = lb.BiontySource.filter(
    entity="DevelopmentalStage", organism="mouse"
).one()

for name, terms in ontology_ids.items():
    accessor, orm = FEATURE_TO_ACCESSOR.get(name)
    terms_ids = [i[1] for i in terms]
    result = orm.inspect(terms_ids, field="ontology_id")
    if len(result.non_validated) > 0:
        if name == "development_stage":
            dv_records = orm.from_values(
                result.non_validated,
                field="ontology_id",
                bionty_source=bionty_source_ds_mouse,
            )
        elif name == "tissue":
            ts_records = [
                orm(name=term[0], ontology_id=term[1])
                for term in terms
                if term[1] in result.non_validated
            ]

ln.save(dv_records)
ln.save(ts_records)

### donors and suspension types

In [ ]:
donor_ids = set()
suspension_types = set()

for i in res_content:
    if "donor_id" in i:
        donor_ids.update(i["donor_id"])
    if "suspension_type" in i:
        suspension_types.update(i["suspension_type"])

In [ ]:
donors = ln.ULabel.filter(name="is_donor").one().children.all()
result = donors.inspect(donor_ids)
new_donors = [ln.ULabel(name=name) for name in result.non_validated]
ln.save(new_donors)
is_donor = ln.ULabel.filter(name="is_donor").one()
is_donor.children.add(*new_donors)

In [ ]:
stypes = ln.ULabel.filter(name="is_suspension_type").one().children.all()
result = stypes.inspect(suspension_types)
new_stypes = [ln.ULabel(name=name) for name in result.non_validated]
ln.save(new_stypes)
is_suspension_type = ln.ULabel.filter(name="is_suspension_type").one()
is_suspension_type.children.add(*new_stypes)

## Annotate files

In [ ]:
for idx, dataset_meta in enumerate(res_content):
    if idx % 50 == 0:
        print(f"annotating dataset {idx} of {len(res_content)}")
    file = files.filter(key__contains=dataset_meta["dataset_id"]).one_or_none()
    if file is None:
        continue
    for field, terms in dataset_meta.items():
        if field not in FEATURE_TO_ACCESSOR:
            continue
        accessor, orm = FEATURE_TO_ACCESSOR.get(field)
        if field in ["donor_id", "suspension_type"]:
            records = orm.from_values(terms, field="name")
        else:
            records = orm.from_values(
                [i["ontology_term_id"] for i in terms], field="ontology_id"
            )
        if len(records) > 0:
            getattr(file, accessor).add(*records)

In [ ]:
files.last().describe()

## Register genes

In [ ]:
ln.settings.track_run_inputs = False
ln.settings.verbosity = "hint"

for idx, file in enumerate(files):
    if idx % 50 == 0:
        print(f"annotating file {idx} of {len(files)}")
    adata_backed = file.backed()
    genes = adata_backed.var_names
    organism = file.organism.first()
    featureset = ln.FeatureSet.from_values(
        genes, field=lb.Gene.ensembl_gene_id, organism=organism
    )
    # skips non-human datasets
    if featureset is None:
        continue
    if featureset._state.adding:
        featureset.save()
    # not sure why some feature sets are not linked with any genes
    if featureset.genes.count() == 0:
        records = lb.Gene.from_values(
            genes, field=lb.Gene.ensembl_gene_id, organism=organism
        )
        featureset.genes.set(records)
    file.feature_sets.add(featureset, through_defaults={"slot": "var"})

### Datasets with human or mouse genes but annotated as other organisms

These files don't have a 'var' featureset:

In [ ]:
from django.db.models import Count

novar_files = files.annotate(c=Count("feature_sets")).filter(c=2).all()
len(novar_files)

In [ ]:
for idx, file in enumerate(novar_files):
    if idx % 5 == 0:
        print(f"annotating file {idx} of {len(novar_files)}")
    adata_backed = file.backed()
    genes = lb.Gene.from_values(
        adata_backed.var_names, field=lb.Gene.ensembl_gene_id, organism="human"
    )
    if len(genes) == 0:
        genes = lb.Gene.from_values(
            adata_backed.var_names, field=lb.Gene.ensembl_gene_id, organism="mouse"
        )
    if len(genes) == len(adata_backed.var_names):
        feature_set = ln.FeatureSet(genes, type="number")
        file.feature_sets.add(featureset, through_defaults={"slot": "var"})

### Datasets with unregistered genes

In [ ]:
n = 52126
file = ln.FeatureSet.filter(n=n).one().files.all().one()
adata_backed = file.backed()
genes = lb.Gene.from_values(
    adata_backed.var_names, field=lb.Gene.ensembl_gene_id, organism="mouse"
)
ln.save(genes)

feature_set = ln.FeatureSet(genes, type="number")
file.feature_sets.add(featureset, through_defaults={"slot": "var"})

ln.FeatureSet.filter(n=n).one().delete()

In [ ]:
n = 52127
file = ln.FeatureSet.filter(n=n).one().files.all().one()
adata_backed = file.backed()
genes = lb.Gene.from_values(
    adata_backed.var_names, field=lb.Gene.ensembl_gene_id, organism="mouse"
)
ln.save(genes)
feature_set = ln.FeatureSet(genes, type="number")
file.feature_sets.add(featureset, through_defaults={"slot": "var"})

ln.FeatureSet.filter(n=n).one().delete()

In [ ]:
n = 39091
for file in ln.FeatureSet.filter(n=n).one().files.all():
    adata_backed = file.backed()
    genes = lb.Gene.from_values(
        adata_backed.var_names, field=lb.Gene.ensembl_gene_id, organism="mouse"
    )
    ln.save(genes)
    feature_set = ln.FeatureSet(genes, type="number")
    file.feature_sets.add(featureset, through_defaults={"slot": "var"})

ln.FeatureSet.filter(n=n).one().delete()

In [ ]:
n = 35412
file = ln.FeatureSet.filter(n=n).one().files.all().one()
adata_backed = file.backed()
genes = lb.Gene.from_values(
    adata_backed.var_names, field=lb.Gene.ensembl_gene_id, organism="mouse"
)
ln.save(genes)
feature_set = ln.FeatureSet(genes, type="number")
file.feature_sets.add(featureset, through_defaults={"slot": "var"})

ln.FeatureSet.filter(n=n).one().delete()

### Register ERCC genes

Register the organism:

In [ ]:
organism_ercc = lb.Organism.from_bionty(
    ontology_id="NCBITaxon:32630", bionty_source=ncbitaxon_bs
)
organism_ercc.save(parents=False)
organism_ercc

Get the gene table from cellxgene:

In [ ]:
import pandas as pd

df_ercc = pd.read_csv(
    "https://github.com/chanzuckerberg/single-cell-curation/raw/main/cellxgene_schema_cli/cellxgene_schema/ontology_files/genes_ercc.csv.gz",
    header=None,
)

In [ ]:
df_ercc

In [ ]:
ercc_genes = []

for _, row in df_ercc.iterrows():
    ercc_genes.append(
        lb.Gene(
            symbol=row[0], stable_id=row[0], description=row[1], organism=organism_ercc
        )
    )

In [ ]:
ln.save(ercc_genes)

### Datasets with multi-organism genes

5 files have ERCC genes

In [ ]:
for file in files.all():
    adata_backed = file.backed()
    var_names = adata_backed.var_names
    if len(var_names[var_names.str.startswith("ERCC")]) > 0:
        print(file)

In [ ]:
for uid in [
    "KuIP3MrldEjc7dPantpo",
    "MeByXegdYg3sdk9vueHh",
    "m0OxWMCH24t2qmZHjUu8",
    "rmrJahYS2l4lUuHyK1hh",
    "5e0KFuZGuR4YMMOePdOT",
]:
    file = files.get(uid=uid)
    adata_backed = file.backed()
    genes_ercc = lb.Gene.filter(organism=organism_ercc).all()
    genes = [i for i in genes_ercc if i.symbol in adata_backed.var_names]
    feature_set_ercc = ln.FeatureSet(genes, type="number")
    feature_set_ercc.save()
    file.feature_sets.add(feature_set_ercc, through_defaults={"slot": "var-ercc"})

In [ ]:
files.annotate(c=Count("feature_sets")).filter(c=4).df()